<a href="https://colab.research.google.com/github/bogdanbabych/a05mteval/blob/main/bleuFileV04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BLEU: BiLingual Evaluation Understudy

*NLP evaluation metric used in Machine Translation tasks*

*Suitable for measuring corpus level similarity*

*$n$-gram comparison between words in candidate sentence and reference sentences*

*Range: 0 (no match) to 1 (exact match)*

### 1. Libraries
*Install and import necessary libraries*


In [ ]:
import nltk
import nltk.translate.bleu_score as bleu

import math
import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### 2. Dataset
*Array of words: candidate and reference sentences split into words*

In [ ]:
hyp = str('EU Commission President von der Leyen promises US scientists financial incentives and a legal obligation to respect their freedom of research.').split()
hypES = str('In an implicit criticism of Donald Trump\'s policies, Ursula von der Leyen said that attacking science is a "gigantic miscalculation."').split()
hypESdeepL = str('In an implicit criticism against Donald Trump\'s policies, Ursula von der Leyen said that attacking science is a "gigantic miscalculation".').split()

ref_a = str('In implicit criticism against Donald Trump\'s policies, Ursula von der Leyen said attacking free and open science was a "gigantic miscalculation".').split()
ref_b = str('In a veiled critique of Donald Trump\'s policies, Ursula von der Leyen stated that undermining free and open science was a "huge mistake".').split()

In [ ]:
file_ref = 'news01en.txt'

with open(file_ref, 'r') as file:
    file_content_ref = file.read()



In [ ]:
file_ref = file_content_ref.split()

In [ ]:
file_ref

In [ ]:
file_test = 'news01de2en-g.txt'

with open(file_test, 'r') as file:
    file_content_test = file.read()

In [ ]:
file_test = file_content_test.split()

In [ ]:
hypESdeepL

['In',
 'an',
 'implicit',
 'criticism',
 'against',
 'Donald',
 "Trump's",
 'policies,',
 'Ursula',
 'von',
 'der',
 'Leyen',
 'said',
 'that',
 'attacking',
 'science',
 'is',
 'a',
 '"gigantic',
 'miscalculation".']

### 3. *Sentence* score calculation
*Compares 1 hypothesis (candidate or source sentence) with 1+ reference sentences, returning the highest score when compared to multiple reference sentences.*

In [ ]:
score_ref_a = bleu.sentence_bleu([file_ref], file_test)
print("Hyp and ref_a : {}".format(score_ref_a))
score_ref_b = bleu.sentence_bleu([ref_b], hypES)
print("Hyp and ref_b : {}".format(score_ref_b))
score_ref_ab = bleu.sentence_bleu([ref_a, ref_b], hypES)
print("Hyp vs multiple refs: {}".format(score_ref_ab))

Hyp and ref_a : 0.5138118428111933
Hyp and ref_b : 0.33024002397118396
Hyp vs multiple refs: 0.47351481676711066


### 4. *Corpus* score calculation
*Compares 1 candidate document with multiple sentence and 1+ reference documents also with multiple sentences.*

* Different than averaging BLEU scores of each sentence, it calculates the score by *"summing the numerators and denominators for each hypothesis-reference(s) pairs before the division"*

In [ ]:
score_ref_a = bleu.corpus_bleu([[ref_a]], [hyp])
print("1 document with 1 reference sentence: {}".format(score_ref_a))
score_ref_a = bleu.corpus_bleu([[ref_a, ref_b]], [hyp])
print("1 document with 2 reference sentences: {}".format(score_ref_a))
score_ref_a = bleu.corpus_bleu([[ref_a], [ref_b]], [hyp, hyp])
print("2 documents with 1 reference sentence each: {}".format(score_ref_a))

1 document with 1 reference sentence: 1.0
1 document with 2 reference sentences: 1.0
2 documents with 1 reference sentence each: 0.8778107713916036


* Cumulative N-grams: *by default, the score is calculatedby considering all $N$-grams equally in a geometric mean*

In [ ]:
score_ngram1 = bleu.sentence_bleu([ref_b], hyp)
score_ngram = bleu.sentence_bleu([ref_b], hyp, weights=(0.25,0.25,0.25,0.25))
score_ngram_geo = (11/11*9/10*6/9*4/8)**0.25
print("N-grams: {} = {} = ".format(score_ngram1, score_ngram, score_ngram_geo))

N-grams: 0.7400828044922853 = 0.7400828044922853 = 


### Further testing

In [ ]:
hyp = str('she read the book because she was interested in world history').split()
ref_a = str('she was interested in world history because she read the book').split()
hyp_b = str('the book she read was about modern civilizations.').split()
ref_b = str('the book she read was about modern civilizations.').split()

score_a = bleu.sentence_bleu([ref_a], hyp)
score_b = bleu.sentence_bleu([ref_b], hyp_b)
score_ab = bleu.sentence_bleu([ref_a], hyp_b)
score_ba = bleu.sentence_bleu([ref_b], hyp)
score_ref_a = bleu.corpus_bleu([[ref_a], [ref_b]], [hyp, hyp_b])
average = (score_a+score_b)/2
corpus = math.pow((11+8)/19 * (9+7)/(17) * (6+6)/(9+6) * (4+5)/(8+5), 1/4)
print("Sent: {}, {}, {}, {} - Corpus {}, {}, {}".format(score_a, score_b, score_ab, score_ba, score_ref_a, average, corpus))

Sent: 0.7400828044922853, 1.0, 6.664457123729399e-155, 8.190757052088229e-155 - Corpus 0.8496988908521796, 0.8700414022461427, 0.8496988908521795


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
